In [32]:
# Prepare data for tokenizer
!python3 tokenizer/prepare-visfd.py \
    ./data/Train.csv \
    --out_dir ./tokenizer

2024-04-27 09:26:03 INFO  WordSegmenter:24 - Loading Word Segmentation model
2024-04-27 09:26:03 INFO  PosTagger:23 - Loading POS Tagging model
2024-04-27 09:26:05 INFO  NerRecognizer:34 - Loading NER model
2024-04-27 09:26:14 INFO  DependencyParser:32 - Loading Dependency Parsing model
Progress: 100%|████████████████████████████| 7786/7786 [00:37<00:00, 209.74it/s]


In [33]:
# Build tokenizer
!python3 tokenizer/build-tokenizer.py \
    ./tokenizer/feedbacks.txt \
    --out_dir ./tokenizer \
    --config_file ./config.json

[00:00:00] Tokenize words                 ██████████████████ 7737     /     7737[00:00:00] Tokenize words                 ██████████████████ 0        /        0
[00:00:00] Count pairs                    ██████████████████ 7737     /     7737
[00:00:00] Compute merges                 ██████████████████ 4640     /     4640


In [34]:
from tokenizers import Tokenizer

tokenizer: Tokenizer = Tokenizer.from_file("./tokenizer/tokenizer.json")

In [1]:
from modules import ViSFD_LSTM
import torch
vocab_size = 5000
N, L = 5, 7
input = torch.randint(0, vocab_size, (N, L))
model = ViSFD_LSTM(vocab_size, task_type="mtl")

o, others = model(input)
o.size(), others.size()

torch.Size([5, 11]) torch.Size([5, 10, 3])


RuntimeError: All Tensors given to nested_tensor must have the same dimension. Found dimension 3 for Tensor at index 1 and dimension 2 for Tensor at index 0.